# Trabajo Final AIVA: Narrador Automático de Ajedrez 

## Descripción del Proyecto
Este proyecto implementa un sistema de visión artificial capaz de **analizar una partida de ajedrez en vídeo y narrar las jugadas en tiempo real**.

El sistema utiliza **YOLOv8** para la detección de piezas y un algoritmo lógico personalizado para interpretar las reglas del ajedrez (movimientos, capturas, enroques y coronaciones), mostrando el historial de la partida en una interfaz gráfica superpuesta.

### Autor
* **Raúl Sánchez Ibáñez**
* Asignatura: Aplicaciones Industriales de Visión Artificial

## 1. Generación del Dataset: Extracción de Frames

Antes de entrenar el modelo, es necesario construir un banco de imágenes. En esta fase, procesamos los vídeos de partidas grabadas para convertirlos en imágenes estáticas que servirán de base para el entrenamiento.

Utilizamos la librería **OpenCV** para leer los archivos de vídeo y extraer fotogramas periódicamente.
* **Estrategia:** Se guarda solo 1 frame cada 150 (variable `saltar_frames`) para garantizar variedad en las posiciones del tablero y evitar tener miles de imágenes casi idénticas, lo cual podría provocar *overfitting* (sobreajuste).
* **Salida:** Las imágenes resultantes se almacenan en la carpeta `dataset`, listas para ser etiquetadas manualmente.

In [9]:
import cv2
import os

# CONFIGURACIÓN 
# Ruta base donde estan los videos

RUTA_VIDEOS = "T05-T06_videos"

# Nombres de los videos
VIDEOS = [
    "Video1.mp4",
    "Video2.mp4"
]

# Carpeta donde se guardarán las fotos extraídas
CARPETA_SALIDA = os.path.join(RUTA_VIDEOS, "dataset")

def extraer_frames(video_filename, saltar_frames=150):
    video_path = os.path.join(RUTA_VIDEOS, video_filename)
    
    # Verificar que el archivo existe antes de intentar abrirlo
    if not os.path.exists(video_path):
        print(f"❌ NO ENCONTRADO: {video_path}")
        print("   -> Verifica si falta la letra de la unidad (ej: 'C:\\...') o si la ruta es correcta.")
        return

    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"❌ Error al abrir el video: {video_filename}")
        return

    # Crear carpeta de salida si no existe
    if not os.path.exists(CARPETA_SALIDA):
        os.makedirs(CARPETA_SALIDA)
        print(f"📂 Carpeta creada: {CARPETA_SALIDA}")

    count = 0
    saved_count = 0
    video_name_simple = os.path.splitext(video_filename)[0][:15] # Usamos un nombre corto para el archivo

    print(f"🎥 Procesando: {video_filename}...")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Guardamos 1 frame cada 'saltar_frames'
        if count % saltar_frames == 0:
            output_name = f"{video_name_simple}_{count}.jpg"
            save_path = os.path.join(CARPETA_SALIDA, output_name)           
            cv2.imwrite(save_path, frame)
            saved_count += 1
        
        count += 1

    cap.release()
    print(f"✅ Terminado. Se guardaron {saved_count} imágenes de este video.\n")





In [ ]:
for video in VIDEOS:
    extraer_frames(video, saltar_frames=150) # 150 frames ~ 5 segundos a 30fps
print(f"🚀 LISTO. Revisa la carpeta: {CARPETA_SALIDA}")

'\nfor video in VIDEOS:\n    extraer_frames(video, saltar_frames=150) # 150 frames ~ 5 segundos a 30fps\nprint(f"🚀 LISTO. Revisa la carpeta: {CARPETA_SALIDA}")\n'

## 2. Etiquetado de Datos

Una vez tenemos las imágenes "crudas", es necesario enseñar a la IA qué es cada cosa. Para ello, utilizamos la herramienta gráfica **LabelImg**.

Este script lanza la aplicación configurándola automáticamente para:
1.  Abrir directamente la carpeta de imágenes extraídas.
2.  **Cargar la lista de clases predefinida (`classes.txt`):** Esto es crucial para asegurar la consistencia. Al pasar el archivo como argumento, garantizamos que el ID `0` siempre sea `p_b` (Peón Blanco), el ID `1` sea `p_n`, etc., evitando que el orden cambie entre sesiones.
3.  Guardar las etiquetas en formato **YOLO** (archivos `.txt` con coordenadas normalizadas).

In [16]:
import os

# 1. Define dónde están las cosas
carpeta_imagenes = "T05-T06_videos/dataset_raw/images" 
archivo_clases = "T05-T06_videos/dataset_raw/labels/classes.txt"                 

# Esto lanza el programa pasándole el archivo de clases como argumento
comando = f"labelImg {carpeta_imagenes} {archivo_clases}"

# 3. Ejecutar
print(f"Ejecutando: {comando}")
os.system(comando)

Ejecutando: labelImg T05-T06_videos/dataset_raw/images T05-T06_videos/dataset_raw/labels/classes.txt


0

## 3. Preparación y Partición del Dataset (Train/Val Split) 

Una vez etiquetadas las imágenes, debemos organizar los datos en la estructura que YOLO necesita (`train` y `val`). En lugar de hacer una división aleatoria simple, aplicamos una **estrategia híbrida inteligente** para maximizar el aprendizaje y obtener métricas honestas:

1.  **Datos Reales (Frames de vídeo):** Se dividen aleatoriamente en **80% Entrenamiento / 20% Validación**. Esto garantiza que el examen final del modelo (la validación) se realice sobre imágenes de partidas reales, que es el objetivo del proyecto.
2.  **Datos Sintéticos (Capturas personalizadas):** Se envían al **100% a Entrenamiento**. Estos datos sirven para que la red neuronal aprenda la morfología de las piezas en situaciones extremas, pero no queremos validarnos con ellos porque distorsionarían las métricas de rendimiento real.

El siguiente script automatiza la creación de la estructura de carpetas, realiza el reparto selectivo y genera el archivo de configuración `data.yaml` necesario para el entrenamiento.

In [12]:
import os
import shutil
import random
import yaml

# CONFIGURACIÓN
# 1. Dónde está todo mezclado ahora
origen_dir = 'T05-T06_videos/dataset_raw'

# 2. Dónde vamos a poner los datos ordenados
destino_dir = 'T05-T06_videos/dataset_ready'

# 3. LISTA DE CLASES
classes_list = [
    "p_b", "p_n", "t_n", "c_n", "a_n", "d_n", 
    "r_n", "t_b", "c_b", "a_b", "d_b", "r_b"
]

def preparar_yolo_inteligente():
    src_images = os.path.join(origen_dir, 'images')
    src_labels = os.path.join(origen_dir, 'labels')

    # Limpieza previa del destino
    if os.path.exists(destino_dir):
        shutil.rmtree(destino_dir)
    
    # Crear carpetas vacías
    for split in ['train', 'val']:
        os.makedirs(os.path.join(destino_dir, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(destino_dir, split, 'labels'), exist_ok=True)

    # Listar todas las imágenes
    todas_imagenes = [f for f in os.listdir(src_images) if f.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp'))]
    
    print(f"🔎 Analizando {len(todas_imagenes)} imágenes en total...")

    # Listas temporales
    datos_videos = []   # Partidas reales
    datos_capturas = [] # Tableros personalizados

    for img_file in todas_imagenes:
        nombre_base = os.path.splitext(img_file)[0]
        txt_file = nombre_base + ".txt"
        txt_path = os.path.join(src_labels, txt_file)

        # Verificamos que tenga etiqueta
        if os.path.exists(txt_path):
            pareja = (img_file, txt_file)
            
            # Clasificamos según el nombre del archivo
            if img_file.startswith("Video"):
                datos_videos.append(pareja)
            else:
                # Asumimos que si no es Video, es Captura de pantalla 
                datos_capturas.append(pareja)
    
    print(f"✅ Detectados:")
    print(f"   - {len(datos_videos)} frames de Vídeo (Partidas Reales)")
    print(f"   - {len(datos_capturas)} capturas personalizadas (Sintéticas)")

    # DIVISIÓN DE DATOS
    
    # 1. Los Videos los mezclamos y dividimos 80/20
    random.seed(42)
    random.shuffle(datos_videos)
    
    split_idx = int(len(datos_videos) * 0.8)
    
    videos_train = datos_videos[:split_idx]
    videos_val = datos_videos[split_idx:]
    
    # 2. Las Capturas van TODAS a train (sin mezclar ni dividir)
    # Así el train tiene variedad, pero el val examina con partidas reales
    train_final = videos_train + datos_capturas
    val_final = videos_val

    print("-" * 30)
    print(f"📊 REPARTO FINAL:")
    print(f"   TRAIN: {len(train_final)} imágenes ({len(videos_train)} reales + {len(datos_capturas)} sintéticas)")
    print(f"   VAL:   {len(val_final)} imágenes (Solo reales)")

    # COPIADO
    def copiar(lista, split):
        print(f"📂 Copiando archivos a {split}...")
        for img, txt in lista:
            shutil.copy(os.path.join(src_images, img), 
                        os.path.join(destino_dir, split, 'images', img))
            shutil.copy(os.path.join(src_labels, txt), 
                        os.path.join(destino_dir, split, 'labels', txt))

    copiar(train_final, 'train')
    copiar(val_final, 'val')

    # YAML
    yaml_content = {
        'path': os.path.abspath(destino_dir),
        'train': 'train/images',
        'val': 'val/images',
        'names': {i: name for i, name in enumerate(classes_list)}
    }

    yaml_path = os.path.join(destino_dir, 'data.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, sort_keys=False)

    print("-" * 30)
    print(f"✨ ¡Dataset listo! Configuración guardada en: {yaml_path}")


    

In [ ]:
preparar_yolo_inteligente()

## 4. Entrenamiento Preliminar: Modelo Base (YOLOv8 Nano)

Para la primera iteración del sistema, seleccionamos la arquitectura **YOLOv8n (Nano)**. Este es el modelo más ligero y rápido de la familia YOLO, ideal para establecer una línea base de rendimiento (baseline) y verificar que el pipeline de datos funciona correctamente.

Configuramos parámetros específicos para el contexto del ajedrez:
* **Resolución Alta (`imgsz=1280`):** Fundamental para distinguir piezas pequeñas (como los peones) en planos generales, ya que la resolución estándar de 640x640 suele ser insuficiente para este dominio.
* **Aumentación de Datos:** Activamos `mosaic` y ligeras rotaciones para que el modelo aprenda a generalizar y no dependa de una alineación perfecta del tablero.

*Nota: Este modelo sirve como prueba inicial. Posteriormente se refinará el entrenamiento utilizando una arquitectura más compleja (Small) para mejorar la precisión en casos difíciles.*

In [ ]:
from ultralytics import YOLO

# 1. Cargar modelo nano
model = YOLO('yolov8n.pt') 

# 2. Iniciar Entrenamiento
results = model.train(
    # Ruta al archivo yaml creado en el paso anterior
    data='T05-T06_videos/dataset_ready/data.yaml',
    
    epochs=100,        # Damos 100 vueltas (se parará antes si deja de aprender)
    imgsz=1280,        # Alta resolución para ver detalles
    batch=4,           # Batch pequeño para no saturar memoria (si falla, baja a 2)
    patience=20,       # Si en 20 épocas no mejora, paramos
    name='ajedrez_final',
    
    # Ajustes extra para mejorar detección
    mosaic=1.0,        # Ayuda a detectar objetos pequeños
    degrees=5.0,       # Rotación ligera para que aprenda tableros torcidos
)

## 5. Entrenamiento Avanzado: Modelo Robusto (YOLOv8 Small) 
Tras validar el flujo de trabajo con el modelo Nano, procedemos al entrenamiento del modelo final utilizando la arquitectura **YOLOv8s (Small)**. Esta versión cuenta con un mayor número de parámetros (11.1M frente a los 3.2M del Nano), lo que le otorga una capacidad superior para distinguir detalles sutiles, como la diferencia entre un Alfil y un Peón en posiciones complejas.

### Estrategia de Aumentación de Datos (Data Augmentation)
Para garantizar que el sistema funcione en condiciones reales y no se vea afectado por las ayudas visuales de las plataformas (ej: casillas que se iluminan en azul o rojo al hacer una "Jugada Brillante"), hemos implementado una configuración de aumentación agresiva:

* **Inmunidad al Color (HSV):** Modificamos intensamente la saturación (`hsv_s=0.7`) y el brillo (`hsv_v=0.4`) durante el entrenamiento. Esto fuerza a la red a aprender las **formas** de las piezas en lugar de depender de sus colores exactos, haciéndola robusta frente a tableros de colores exóticos o cambios de iluminación.
* **Geometría:** Aumentamos la rotación (`degrees=10.0`) y el escalado (`scale=0.5`) para que el detector sea invariante a la posición de la cámara o al zoom del navegador.
* **Duración:** Extendemos el entrenamiento a **120 épocas** con un `patience` de 25, permitiendo que el modelo converja completamente sin detenerse prematuramente ante estancamientos temporales.

In [ ]:
from ultralytics import YOLO

# CAMBIO 1: Cargamos el modelo "Small" (s)
# Es el hermano mayor del Nano. Más preciso, pero requiere un poco más de GPU.
model = YOLO('yolov8s.pt') 

# 2. Iniciar Entrenamiento
results = model.train(
    data='T05-T06_videos/dataset_ready/data.yaml',
    
    # Parámetros de Entrenamiento
    epochs=120,        # Subimos un poco las épocas
    patience=25,       # Más paciencia
    imgsz=1280,        # Mantenemos la alta resolución
    batch=4,           
    
    # CAMBIO 2: Nombre nuevo
    name='ajedrez_pro_small', 
    
    # Aumentación de Datos (Data Augmentation)
    # Esto crea "alucinaciones" durante el entrenamiento para hacerlo robusto
    
    mosaic=1.0,        # Mantenemos el mosaico (bueno para objetos pequeños)
    degrees=10.0,      # Subimos un poco la rotación (para cámaras movidas)
    scale=0.5,         # Zoom in/out (para que entienda piezas cerca y lejos)
    
    # CAMBIO 3: Inmunidad a los colores (Jugada Brillante / Tableros Raros)
    hsv_h=0.015,       # Cambia ligeramente el tono de color
    hsv_s=0.7,         # Cambia MUCHO la saturación (colores vivos vs apagados)
    hsv_v=0.4,         # Cambia el brillo (luz vs oscuridad)
)

## 6. Configuración del Sistema de Inferencia y Lógica Heurística 

Una vez entrenado el modelo, pasamos a la fase de **explotación**. En este bloque definimos las constantes globales que gobernarán el comportamiento del sistema durante el procesamiento del vídeo.

Cabe destacar dos parámetros críticos para la estabilidad del sistema:
* **`SKIP_FRAMES = 3`:** Optimización de rendimiento. No procesamos cada frame individualmente (lo cual sería redundante y lento), sino uno de cada 3. Esto triplica la velocidad de procesamiento sin perder información relevante, ya que el ajedrez es un juego de movimientos pausados.
* **`UMBRAL_CONFIRMACION = 5`:** Filtro de **estabilidad temporal**. Para evitar falsos positivos (por ejemplo, detectar una pieza fantasma mientras la mano del jugador cruza el tablero), el sistema exige que una nueva posición del tablero se mantenga idéntica durante 5 comprobaciones seguidas antes de registrar la jugada oficialmente.

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# RUTAS Y PARÁMETROS 
MODEL_PATH1 = "runs/detect/ajedrez_final/weights/best.pt"
MODEL_PATH = "runs/detect/ajedrez_pro_small/weights/best.pt"

# Configuración visual
ANCHO_SIDEBAR = 350
COLOR_FONDO = 50 
SKIP_FRAMES = 3 
# Procesamos 1 de cada 3 frames para triplicar la velocidad
# sin perder precisión, ya que el ajedrez es lento.

# PARÁMETRO DE ESTABILIDAD
# Filtro temporal: La jugada debe mantenerse estable 5 veces
# para descartar el movimiento de la pieza.
UMBRAL_CONFIRMACION = 5

## 7. Implementación del Núcleo: Lógica de Juego y Motor de Renderizado

En esta sección se implementa el corazón del sistema, estructurado en tres componentes principales que transforman las detecciones visuales en una narración coherente:

### 1. Clase `TableroInteligente` (Calibración Espacial)
Es la encargada de traducir los píxeles de la imagen a coordenadas de ajedrez (a1-h8).
* **Auto-Calibración Segura:** Implementa un mecanismo de seguridad al inicio (`calibrar`). El sistema no define los bordes del tablero hasta que detecta **exactamente 32 piezas** de forma estable durante varios frames. Esto evita que el tablero se deforme si el vídeo empieza con un fundido o una transición.
* **Mapeo Dinámico:** Calcula las dimensiones de las casillas en función de los extremos detectados (`min_x`, `max_x`, etc.), permitiendo que el sistema funcione independientemente del tamaño del tablero en la pantalla.

### 2. Clase `Narrador` (Motor de Reglas)
Gestiona el estado de la partida y aplica la lógica de ajedrez:
* **Interpretación de Movimientos:** Compara el estado actual con el anterior para deducir qué ha ocurrido.
    * **Movimiento Simple:** Una pieza desaparece de A y aparece en B.
    * **Captura:** Dos piezas desaparecen (atacante y víctima) y una aparece (atacante en destino).
    * **Enroque:** Detecta el movimiento simultáneo de Rey y Torre.
    * **Coronación:** Detecta si un Peón desaparece y aparece una pieza mayor (Dama, Torre...) en la misma jugada.
* **Filtro de Estabilidad:** Utiliza un contador (`contador_estabilidad`) para ignorar detecciones erráticas o el movimiento de la mano, confirmando la jugada solo cuando el tablero se queda quieto.

### 3. Procesamiento de Vídeo y UI (`procesar_video_con_sidebar`)
Es el bucle principal que orquesta la ejecución:
1.  Obtiene las detecciones de YOLO frame a frame.
2.  Alimenta al `Narrador` con los datos.
3.  **Visualización (Overlay):** Dibuja una interfaz gráfica avanzada sobre el vídeo original. Se ha diseñado un **panel lateral semitransparente** (sidebar) que se posiciona dinámicamente a la derecha del tablero, asegurando que la información (historial y estado) sea legible sin ocultar la acción del juego.

In [ ]:
class TableroInteligente:
    def __init__(self):
        self.min_x, self.min_y = float('inf'), float('inf')
        self.max_x, self.max_y = float('-inf'), float('-inf')
        self.calibrado = False
        self.frames_validos = 0
        self.LIMITE_CALIBRACION = 20
        
        # Seguridad de Inicio
        self.MIN_PIEZAS_PARA_ACTIVAR = 32   # Exigimos el tablero completo
        self.racha_estabilidad_inicio = 0   # Contador de frames perfectos
        self.FRAMES_PARA_FIARSE = 3        # Frames seguidos con 32 piezas para empezar

    def calibrar(self, detecciones):
        cantidad = len(detecciones)
        
        # 1. Filtro de Estabilidad:
        # Exigimos ver las 32 piezas exactas. Si falta una o hay un falso positivo,
        # asumimos que el tablero no es fiable (transición de vídeo).
        if cantidad != self.MIN_PIEZAS_PARA_ACTIVAR:
            self.racha_estabilidad_inicio = 0
            return False
        
        # Si vemos 32, sumamos confianza
        self.racha_estabilidad_inicio += 1
        
        # Si aún no llevamos suficientes frames estables, esperamos.
        if self.racha_estabilidad_inicio < self.FRAMES_PARA_FIARSE:
            return False

        # A PARTIR DE AQUÍ EL TABLERO ES SEGURO
        # Solo entramos aquí si llevamos 10 frames seguidos viendo 32 piezas
        
        x1s = [box[0] for box in detecciones]
        y1s = [box[1] for box in detecciones]
        x2s = [box[2] for box in detecciones]
        y2s = [box[3] for box in detecciones]

        self.min_x = min(self.min_x, min(x1s))
        self.min_y = min(self.min_y, min(y1s))
        self.max_x = max(self.max_x, max(x2s))
        self.max_y = max(self.max_y, max(y2s))
        
        self.frames_validos += 1
        if self.frames_validos > self.LIMITE_CALIBRACION:
            self.calibrado = True
            
        return True # Retornamos True para indicar que estamos "viendo" el tablero válido

    def obtener_casilla(self, bbox):
        if not self.calibrado: return None
        x1, y1, x2, y2 = bbox
        cx, cy = (x1 + x2) / 2, (y1 + y2) / 2
        ancho, alto = self.max_x - self.min_x, self.max_y - self.min_y
        
        if ancho == 0 or alto == 0: return None
        
        margen = 40
        if (cx < self.min_x - margen or cx > self.max_x + margen or 
            cy < self.min_y - margen or cy > self.max_y + margen): return None

        col = int(((cx - self.min_x) / ancho) * 8)
        fila = int(8 - ((cy - self.min_y) / alto) * 8)
        col = max(0, min(7, col))
        fila = max(0, min(7, fila))
        letras = ['a','b','c','d','e','f','g','h']
        return f"{letras[col]}{fila + 1}"

class Narrador:
    def __init__(self):
        self.tablero = TableroInteligente()
        self.estado_anterior = {} 
        self.historial = [] 
        self.nombres = {
            'p_b': 'Peon B', 't_b': 'Torre B', 'c_b': 'Caballo B', 'a_b': 'Alfil B', 'd_b': 'Dama B', 'r_b': 'Rey B',
            'p_n': 'Peon N', 't_n': 'Torre N', 'c_n': 'Caballo N', 'a_n': 'Alfil N', 'd_n': 'Dama N', 'r_n': 'Rey N'
        }
        self.estado_sistema = "Esperando..."
        
        self.jugada_candidata = None
        self.contador_estabilidad = 0

    def procesar(self, detecciones):
        if not self.tablero.calibrado:
            viendo = self.tablero.calibrar(detecciones)
            # Mostramos info de progreso
            if seeing := self.tablero.racha_estabilidad_inicio > 0:
                self.estado_sistema = f"Estabilizando ({self.tablero.racha_estabilidad_inicio}/{self.tablero.FRAMES_PARA_FIARSE})..."
            else:
                self.estado_sistema = "Esperando tablero completo (32)..."
            
            if self.tablero.frames_validos > 0:
                 self.estado_sistema = "Calibrando dimensiones..."
            return

        estado_actual = {}
        for box in detecciones:
            casilla = self.tablero.obtener_casilla(box[:4])
            if casilla: estado_actual[casilla] = box[4]

        if not self.estado_anterior:
            self.estado_anterior = estado_actual
            self.estado_sistema = "Partida iniciada"
            return

        # Detección de Cambios
        desap = [(k, v) for k, v in self.estado_anterior.items() if k not in estado_actual]
        apar = [(k, v) for k, v in estado_actual.items() if k not in self.estado_anterior or self.estado_anterior[k] != v]
        
        hay_cambio = len(desap) > 0 or len(apar) > 0
        texto_jugada = self._interpretar_jugada(desap, apar)

        # Lógica de Buffer
        if hay_cambio:
            # Creamos una firma única del cambio convirtiendo las listas a texto.
            # Si el frame siguiente tiene la misma firma, es que el tablero está quieto.
            firma_cambio = str(desap) + str(apar)
            
            if firma_cambio == self.jugada_candidata:
                self.contador_estabilidad += 1
            else:
                # Si la firma cambia, reseteamos el análisis
                self.jugada_candidata = firma_cambio
                self.contador_estabilidad = 1
                self.estado_sistema = "Analizando..."
            
            if self.contador_estabilidad >= UMBRAL_CONFIRMACION:
                if texto_jugada:
                    self.historial.append(texto_jugada)
                    self.estado_sistema = "Jugando"
                else:
                    print("⚠️ Movimiento complejo. Actualizando estado.")

                self.estado_anterior = estado_actual 
                self.jugada_candidata = None
                self.contador_estabilidad = 0
        else:
            if self.contador_estabilidad > 0:
                self.contador_estabilidad -= 1

    def _interpretar_jugada(self, desap, apar):
        # 1. Movimiento (1 sale, 1 llega)
        if len(desap) == 1 and len(apar) == 1:
            orig_c, orig_p = desap[0]
            dest_c, dest_p = apar[0]
            
            nombre_origen = self.nombres.get(orig_p, orig_p)
            
            if orig_p == dest_p:
                return f"{nombre_origen}: {orig_c} -> {dest_c}"
            
            # Coronacion
            elif 'p_' in orig_p: 
                nombre_nuevo = self.nombres.get(dest_p, dest_p)
                return f"{nombre}: {cas} -> {dest_c} ({nombre_nuevo})"

        # 2. Captura (2 salen, 1 llega)
        elif len(desap) == 2 and len(apar) == 1:
             dest_c, dest_p = apar[0]
             
             # Captura Normal
             for cas, pza in desap:
                 if pza == dest_p: 
                     nombre = self.nombres.get(pza, pza)
                     return f"{nombre}: {cas} -> {dest_c}"
            
             # Captura con Promoción
             for cas, pza in desap:
                 if 'p_' in pza: 
                     nombre_origen = self.nombres.get(pza, pza)
                     nombre_nuevo = self.nombres.get(dest_p, dest_p)
                     return f"{nombre}: {cas} -> {dest_c} ({nombre_nuevo})"
        
        # 3. ENROQUES
        elif len(desap) == 2 and len(apar) == 2:
            for casilla, pieza in apar:
                if pieza == 'r_b':
                    if casilla == 'g1': return "Enroque Corto Blanco"
                    if casilla == 'c1': return "Enroque Largo Blanco"
                if pieza == 'r_n':
                    if casilla == 'g8': return "Enroque Corto Negro"
                    if casilla == 'c8': return "Enroque Largo Negro"
            return "Enroque" 

        return None


def procesar_video_con_sidebar(VIDEO_ENTRADA, VIDEO_SALIDA):
    print(f"Cargando modelo: {MODEL_PATH}")
    model = YOLO(MODEL_PATH)
    cap = cv2.VideoCapture(VIDEO_ENTRADA)
    
    if not cap.isOpened():
        print(f"❌ Error al abrir el vídeo: {VIDEO_ENTRADA}")
        return

    frame_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    out_w = frame_w 
    out_h = frame_h

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(VIDEO_SALIDA, fourcc, fps, (out_w, out_h))

    narrador = Narrador()
    frame_count = 0
    
    # Configuración del panel
    ANCHO_PANEL = 320   
    MARGEN_TABLERO = 50 
    
    print("🎬 Procesando vídeo... (Pulsa 'q' para salir antes)")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break

        # 1. Detección
        results = model.predict(frame, conf=0.5, verbose=False)
        detecciones = []
        for box in results[0].boxes:
            coords = box.xyxy[0].tolist()
            cls = int(box.cls[0])
            label = model.names[cls]
            detecciones.append(coords + [label])

        # 2. Lógica
        if frame_count % SKIP_FRAMES == 0:
            narrador.procesar(detecciones)

        # 3. Dibujado
        frame_pintado = results[0].plot()
        
        t = narrador.tablero
        if t.calibrado:
            # Rectángulo debug
            cv2.rectangle(frame_pintado, (int(t.min_x), int(t.min_y)), (int(t.max_x), int(t.max_y)), (255, 0, 0), 2)
            # Posición dinámica
            x_panel = int(t.max_x) + MARGEN_TABLERO
        else:
            x_panel = frame_w - ANCHO_PANEL - 50

        # Tope de seguridad
        if x_panel + ANCHO_PANEL > frame_w:
            x_panel = frame_w - ANCHO_PANEL

        # OPACIDAD
        overlay = frame_pintado.copy()
        cv2.rectangle(overlay, (x_panel, 0), (x_panel + ANCHO_PANEL, frame_h), (0, 0, 0), -1)
        
        # 0.8 del Overlay (Negro) + 0.2 de la Imagen original = Efecto cristal oscuro
        frame_pintado = cv2.addWeighted(overlay, 0.8, frame_pintado, 0.2, 0)

        # Textos
        x_txt = x_panel + 20
        y_txt = 50
        
        cv2.putText(frame_pintado, "HISTORIAL", (x_txt, y_txt), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        y_txt += 35
        
        color_estado = (0, 255, 0) if "Jugando" in narrador.estado_sistema else (100, 100, 255)
        cv2.putText(frame_pintado, f"[{narrador.estado_sistema}]", (x_txt, y_txt), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_estado, 1)
        y_txt += 40
        
        ultimas = narrador.historial[-15:]
        total_jugadas = len(narrador.historial)
        offset_jugada = max(0, total_jugadas - 15)

        for i, jugada in enumerate(ultimas):
            color = (0, 255, 255) if i == len(ultimas) - 1 else (220, 220, 220)
            numero = offset_jugada + i + 1
            cv2.putText(frame_pintado, f"{numero}. {jugada}", (x_txt, y_txt), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 1)
            y_txt += 30

        # Mostrar ventana
        try:
            cv2.imshow('Narrador Chess', frame_pintado)
            if cv2.waitKey(1) & 0xFF == ord('q'): 
                break
        except Exception:
            pass

        out.write(frame_pintado)
        frame_count += 1

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"✅ ¡Terminado! Guardado en: {VIDEO_SALIDA}")

## 8. Ejecución y Validación Experimental: Procesamiento por Lotes 

Como paso final, ponemos a prueba la robustez del sistema ejecutando el pipeline completo sobre nuestro conjunto de vídeos de prueba (`Video1` a `Video5`).

En este bloque se realizan las llamadas secuenciales a la función principal `procesar_video_con_sidebar`. Para cada caso:
1.  Se carga el vídeo original.
2.  El sistema se **auto-calibra** detectando el tablero inicial (esperando a las 32 piezas).
3.  Se procesa la partida jugada a jugada, generando el historial en tiempo real.
4.  Se exporta un nuevo archivo de vídeo (`partidaX.mp4`) que incluye la visualización de las detecciones y el **panel de narrativa superpuesto**.

La ejecución por lotes nos permite verificar que el sistema es capaz de generalizar y funcionar correctamente en diferentes partidas, manteniendo la estabilidad tanto en aperturas como en finales de juego.

In [ ]:
# Llamamos a la función
procesar_video_con_sidebar("T05-T06_videos/Video1.mp4","T05-T06_videos/partida1.mp4" )
procesar_video_con_sidebar("T05-T06_videos/Video2.mp4","T05-T06_videos/partida2.mp4" )
procesar_video_con_sidebar("T05-T06_videos/Video3.mp4","T05-T06_videos/partida3.mp4" )
procesar_video_con_sidebar("T05-T06_videos/Video4.mp4","T05-T06_videos/partida4.mp4" )
procesar_video_con_sidebar("T05-T06_videos/Video5.mp4","T05-T06_videos/partida5.mp4" )


Cargando modelo: runs/detect/ajedrez_pro_small/weights/best.pt
🎬 Procesando vídeo... (Pulsa 'q' para salir antes)
⚠️ Movimiento complejo. Actualizando estado.
⚠️ Movimiento complejo. Actualizando estado.
✅ ¡Terminado! Guardado en: T05-T06_videos/partida1.mp4
Cargando modelo: runs/detect/ajedrez_pro_small/weights/best.pt
🎬 Procesando vídeo... (Pulsa 'q' para salir antes)
⚠️ Movimiento complejo. Actualizando estado.
✅ ¡Terminado! Guardado en: T05-T06_videos/partida2.mp4
Cargando modelo: runs/detect/ajedrez_pro_small/weights/best.pt
🎬 Procesando vídeo... (Pulsa 'q' para salir antes)
⚠️ Movimiento complejo. Actualizando estado.
⚠️ Movimiento complejo. Actualizando estado.
✅ ¡Terminado! Guardado en: T05-T06_videos/partida3.mp4
Cargando modelo: runs/detect/ajedrez_pro_small/weights/best.pt
🎬 Procesando vídeo... (Pulsa 'q' para salir antes)
⚠️ Movimiento complejo. Actualizando estado.
✅ ¡Terminado! Guardado en: T05-T06_videos/partida4.mp4
Cargando modelo: runs/detect/ajedrez_pro_small/weights